In [80]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string


import tweepy
import pickle
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import twitter
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tania\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [81]:
# Data from TFM_fake_news github
dfpredict = pd.read_csv("Map1.csv",encoding= 'unicode_escape', sep=";")
dfpredict.head()

,topic,text,headline,words_h,word_size_h,avg_syllables_word_h,unique_words_h,ttr_h,mltd_h,sents,...,propn_ratio,noun_ratio,adp_ratio,det_ratio,punct_ratio,pron_ratio,verb_ratio,adv_ratio,sym_ratio,label
0,Science,la nasa recupera el contacto con un satélite d...,la nasa recupera el contacto con un satélite d...,16,538,25,875,9375,7168,16,...,48,2443,1775,1566,71,167,835,313,0,1
1,Economy,amlo aceleraría el consumo y el crecimiento ec...,amlo aceleraría el consumo y el crecimiento ec...,11,527,255,8182,9091,3388,5,...,922,2282,1553,165,1117,49,631,97,0,1
2,Sport,al borde de un colapso nervioso quedó el hábil...,compañero de james se calvea y le juega pesa...,12,375,167,1000,1000,0,13,...,734,1658,1522,1332,1304,435,815,353,54,0
3,Politics,"mediante pupitrazo de último minuto anoche, el...",dian gravará este año a los niños que recojan ...,12,458,192,1000,1000,0,8,...,223,223,1933,1524,1004,74,669,335,0,0
4,Politics,muy temprano esta mañana el expresidente y aho...,uribe asegura que insultó 358 guerrilleros,6,617,25,1000,1000,0,9,...,577,1731,846,1077,200,538,1038,50,77,0


In [82]:
df1 = dfpredict.iloc[:500]
df1

,topic,text,headline,words_h,word_size_h,avg_syllables_word_h,unique_words_h,ttr_h,mltd_h,sents,...,propn_ratio,noun_ratio,adp_ratio,det_ratio,punct_ratio,pron_ratio,verb_ratio,adv_ratio,sym_ratio,label
0,Science,la nasa recupera el contacto con un satélite d...,la nasa recupera el contacto con un satélite d...,16,538,25,875,9375,7168,16,...,48,2443,1775,1566,71,167,835,313,0,1
1,Economy,amlo aceleraría el consumo y el crecimiento ec...,amlo aceleraría el consumo y el crecimiento ec...,11,527,255,8182,9091,3388,5,...,922,2282,1553,165,1117,49,631,97,0,1
2,Sport,al borde de un colapso nervioso quedó el hábil...,compañero de james se calvea y le juega pesa...,12,375,167,1000,1000,0,13,...,734,1658,1522,1332,1304,435,815,353,54,0
3,Politics,"mediante pupitrazo de último minuto anoche, el...",dian gravará este año a los niños que recojan ...,12,458,192,1000,1000,0,8,...,223,223,1933,1524,1004,74,669,335,0,0
4,Politics,muy temprano esta mañana el expresidente y aho...,uribe asegura que insultó 358 guerrilleros,6,617,25,1000,1000,0,9,...,577,1731,846,1077,200,538,1038,50,77,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Politics,el 57% de catalanes cree que puigdemont ganó e...,el 57% de catalanes cree que puigdemont ganó e...,18,428,167,6667,7778,2268,14,...,807,146,1472,1613,1165,218,743,282,0,1
496,Politics,las fuerzas leales al gobierno de sudán del su...,amnistía acusa a las fuerzas del gobierno surs...,16,469,20,875,9375,7168,28,...,525,2059,1796,1453,106,121,928,242,4,1
497,Politics,la consejera de fomento del gobierno de castil...,"fomento pide ""un cambio de rumbo"" en la políti...",14,393,164,8571,9286,5488,12,...,482,1969,1995,1343,952,196,926,30,0,1
498,Entertainment,andrea legarreta explota al recordar polémica ...,andrea legarreta explota al recordar polémica ...,8,60,262,1000,1000,0,15,...,439,152,1257,936,1257,906,1667,614,0,1


In [83]:
#### Creating a function to convert the text in lowercase, remove the extra space, special chr., ulr and links.


In [84]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [85]:
dfpredict["text"] = dfpredict["text"].apply(wordopt)

In [86]:
x = dfpredict["text"]
y = dfpredict["label"]

In [87]:
#### Splitting the dataset into training set and testing set. 

In [88]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state= 8)

In [89]:
#### Convert text to vectors

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [91]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [92]:
### 1. Logistic Regression

In [93]:
from sklearn.linear_model import LogisticRegression

In [94]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [95]:
pred_lr=LR.predict(xv_test)

In [96]:
LR.score(xv_test, y_test)

0.9195278969957081

In [97]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92       471
           1       0.92      0.92      0.92       461

    accuracy                           0.92       932
   macro avg       0.92      0.92      0.92       932
weighted avg       0.92      0.92      0.92       932



In [98]:
### 2. Decision Tree Classification

In [99]:
from sklearn.tree import DecisionTreeClassifier

In [100]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [101]:
pred_dt = DT.predict(xv_test)

In [102]:
DT.score(xv_test, y_test)

0.8969957081545065

In [103]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.90      0.89      0.90       471
           1       0.89      0.90      0.90       461

    accuracy                           0.90       932
   macro avg       0.90      0.90      0.90       932
weighted avg       0.90      0.90      0.90       932



In [104]:
### 3. Gradient Boosting Classifier

In [105]:
from sklearn.ensemble import GradientBoostingClassifier

In [106]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [107]:
pred_gbc = GBC.predict(xv_test)

In [108]:
GBC.score(xv_test, y_test)

0.944206008583691

In [109]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       0.97      0.92      0.94       471
           1       0.92      0.97      0.95       461

    accuracy                           0.94       932
   macro avg       0.95      0.94      0.94       932
weighted avg       0.95      0.94      0.94       932



In [110]:
### 4. Random Forest Classifier

In [111]:
from sklearn.ensemble import RandomForestClassifier

In [112]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [113]:
pred_rfc = RFC.predict(xv_test)

In [114]:
RFC.score(xv_test, y_test)

0.9495708154506438

In [115]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.93      0.97      0.95       471
           1       0.97      0.93      0.95       461

    accuracy                           0.95       932
   macro avg       0.95      0.95      0.95       932
weighted avg       0.95      0.95      0.95       932



In [116]:
# Model Testing With Manual Entry

### News

In [117]:
import pyttsx3 
s = pyttsx3.init()
data = "Fake New"
data1 = "Not A Fake New"  
def output_lable(n):
    if n == 0:
        
        s.say(data)  
        s.runAndWait()
        return "Fake News"
    elif n == 1:
             
        s.say(data1)  
        s.runAndWait() 
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)
    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]), 
                                                                                                              output_lable(pred_DT[0]), 
                                                                                                              output_lable(pred_GBC[0]), 
                                                                                                              output_lable(pred_RFC[0])))
    
                               

In [119]:
try:
    news = str(text_entry_var.get())
    manual_testing(news)
except:
    news = str(input())
    manual_testing(news)


está demostrado que el único  narco estado  fue el régimen que asaltó el poder con el golpe de estado  la derecha mentirosa  famosa por sus  narcovínculos  y  narcoaviones  ni sus aliados sometidos a la dea  tienen moral para criticarnos   bolivia es libre  digna y soberana 


LR Prediction: Fake News 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RFC Prediction: Fake News
